In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.metrics import classification_report

In [2]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.1 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [4]:
#Mount on drive
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [5]:
%cd /content/gdrive/MyDrive/UMN_CS_PhD/RA/Datasets for JOUR8500/GroundTruth/

/content/gdrive/MyDrive/UMN_CS_PhD/RA/Datasets for JOUR8500/GroundTruth


In [6]:
%pwd

'/content/gdrive/MyDrive/UMN_CS_PhD/RA/Datasets for JOUR8500/GroundTruth'

In [7]:
#Read in the most political user_history ( from the other ipynb notebook) + user screen name (We scraped the last 200 tweets of this user, found the most political 10 tweets and chose those)
user_history_path = "most_pol_user_history_CSA.csv"
user_history = pd.read_csv(user_history_path, header=0)
user_history.head()

,user_id,user_history
0,KingB8erXXX,@TLadstrong Fuck YEAH bro sorry that big goobe...
1,etalkCTV,CTV is looking for single Canadians ready to l...
2,hereIcomeagain7,Reports of firing at LAC that borders India &a...
3,valiaek,RT @CNN: The smell of burnt and unburnt mariju...
4,NotoriousVOG,RT @RWwatchMA: MA voters are better informed t...


In [8]:
user_history.shape

(7068, 2)

In [9]:
user_history.rename(columns = {'user_id':'user_screen_name'}, inplace = True)

In [10]:
user_history.head()

,user_screen_name,user_history
0,KingB8erXXX,@TLadstrong Fuck YEAH bro sorry that big goobe...
1,etalkCTV,CTV is looking for single Canadians ready to l...
2,hereIcomeagain7,Reports of firing at LAC that borders India &a...
3,valiaek,RT @CNN: The smell of burnt and unburnt mariju...
4,NotoriousVOG,RT @RWwatchMA: MA voters are better informed t...


### Load model for the political ideology classifier  ###

In [11]:
#Create framework for model
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [12]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
#l = tf.keras.layers.Dropout(1.0, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(outputs['pooled_output'])

# Use inputs and outputs to construct a final model
load_model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [13]:
#load weights
weights_path = "/content/gdrive/MyDrive/UMN_CS_PhD/RA/Datasets for JOUR8500/GroundTruth/usercontext_weights.h5"
load_model.load_weights(weights_path)

### Now run it on Hao's extended data ###

In [14]:
# Read in the tweets from the event (original scraped data which is joined)
event_history_path = "hao_test_labels.csv"
event_history = pd.read_csv(event_history_path, header=0)
event_history.head()

,Embedded_text,Emojis,Timestamp,UserName,UserScreenName,file,user_screen_name,user_id,user_name,description,truncated,created_at,full_text
0,has hobby lobby shut down for this corona stuf...,NaN,2020-03-21T04:49:30.000Z,@rachelspires_,rach,corona AND hobby lobby,rachelspires_,2639724316,rach,girl mom ✨,False,Tue Apr 05 03:44:29 +0000 2022,RT @dearra: I want to go on a food tour. Trave...
1,Companies need to do things correctly during t...,😬 🤡,2020-03-22T04:33:39.000Z,@gabby_diamante,Gabriella Diamante de la Rosa,corona AND hobby lobby,gabby_diamante,44444748,Gabriella Diamante de la Rosa,my personality shines bright and upfront on my...,False,Sun Apr 03 13:19:22 +0000 2022,@NICKIMINAJ Girl don’t play with me ! Talking ...
2,#hobbylobby might help spread the corona virus...,NaN,2020-03-22T03:32:10.000Z,@GiBowXO,GBow,corona AND hobby lobby,GiBowXO,2955828382,GBow,NaN,False,Thu Dec 23 15:51:03 +0000 2021,RT @ISurveyMyDomain: @AZ_Brittney @RexChapman ...
3,Corona fake god told the Hobby Lobby guy's wif...,NaN,2020-03-22T01:41:31.000Z,@kgthatboy,KG,corona AND hobby lobby,kgthatboy,856715778099302401,KG,I'm worst at what I do best,False,Thu Apr 07 23:24:53 +0000 2022,@Onomatopathetic Damn might be a good idea I'l...
4,I wouldn't shop at Hobby Lobby if they were th...,NaN,2020-03-22T20:29:35.000Z,@jaguar_mick,Mick Jaguar,corona AND hobby lobby,jaguar_mick,970013930318802944,Mick Jaguar,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,False,Thu Apr 07 04:43:49 +0000 2022,"The birds and the ladybugs, right @LindseyGrah..."


In [15]:
event_history.shape

(8677, 13)

In [16]:
event_history.rename(columns = {'user_id':'id'}, inplace = True)

In [17]:
# counting unique values
n = len(pd.unique(event_history['id']))
print(n)

7855


In [18]:
#additional_user information like verified tweets (which is from the previous round of run where we just used description)
additional_history_path = "verification_info_hao_test_tweets.csv"
add_history = pd.read_csv(additional_history_path, header=0)
add_history.head()

,id,screen_name,tweet,verified,description
0,1068507930004078593,00lineCAKE,RT @backsdoor: billboard themselves with 13m f...,False,until the last step out🤘🏼|stay|atiny|myday||en...
1,805675496847605760,00RaiserXN,@seidosw like they don’t get the hollywood tre...,False,pretend this is a gundam account - she/her
2,1598986062,00_bart,el rector de la Universidad de Guanajuato y el...,False,a veces hago eco de algunas cosas trascendentes
3,1256295041024262146,05Klub,@DaveDuniversal @scioarchy @GoHeels @UNC_Baske...,False,Ambitious\nFamily man\nBrother\nGrandson\nHusb...
4,1136146465410228224,0Robotcc2,New account is @ccwandbot https://t.co/i99sJ8oGSK,False,bot account for @ccwandco


In [19]:
add_history.rename(columns = {'screen_name':'user_screen_name'}, inplace = True)

In [20]:
add_history.head()

,id,user_screen_name,tweet,verified,description
0,1068507930004078593,00lineCAKE,RT @backsdoor: billboard themselves with 13m f...,False,until the last step out🤘🏼|stay|atiny|myday||en...
1,805675496847605760,00RaiserXN,@seidosw like they don’t get the hollywood tre...,False,pretend this is a gundam account - she/her
2,1598986062,00_bart,el rector de la Universidad de Guanajuato y el...,False,a veces hago eco de algunas cosas trascendentes
3,1256295041024262146,05Klub,@DaveDuniversal @scioarchy @GoHeels @UNC_Baske...,False,Ambitious\nFamily man\nBrother\nGrandson\nHusb...
4,1136146465410228224,0Robotcc2,New account is @ccwandbot https://t.co/i99sJ8oGSK,False,bot account for @ccwandco


In [21]:
add_history.shape

(7858, 5)

In [22]:
#join all three dataframes on screen_name
joined = pd.merge(event_history, user_history, how='inner', on='user_screen_name')
joined = pd.merge(joined, add_history, how='inner', on='user_screen_name')

In [23]:
joined.shape

(7200, 18)

In [24]:
joined.head()

,Embedded_text,Emojis,Timestamp,UserName,UserScreenName,file,user_screen_name,id_x,user_name,description_x,truncated,created_at,full_text,user_history,id_y,tweet,verified,description_y
0,has hobby lobby shut down for this corona stuf...,NaN,2020-03-21T04:49:30.000Z,@rachelspires_,rach,corona AND hobby lobby,rachelspires_,2639724316,rach,girl mom ✨,False,Tue Apr 05 03:44:29 +0000 2022,RT @dearra: I want to go on a food tour. Trave...,RT @Closj_: bad bitches eat pickles|RT @mayowa...,2639724316,RT @dearra: I want to go on a food tour. Trave...,False,girl mom ✨
1,Companies need to do things correctly during t...,😬 🤡,2020-03-22T04:33:39.000Z,@gabby_diamante,Gabriella Diamante de la Rosa,corona AND hobby lobby,gabby_diamante,44444748,Gabriella Diamante de la Rosa,my personality shines bright and upfront on my...,False,Sun Apr 03 13:19:22 +0000 2022,@NICKIMINAJ Girl don’t play with me ! Talking ...,When someone calls you a hater for stating fac...,44444748,@NICKIMINAJ Girl don’t play with me ! Talking ...,False,my personality shines bright and upfront on my...
2,#hobbylobby might help spread the corona virus...,NaN,2020-03-22T03:32:10.000Z,@GiBowXO,GBow,corona AND hobby lobby,GiBowXO,2955828382,GBow,NaN,False,Thu Dec 23 15:51:03 +0000 2021,RT @ISurveyMyDomain: @AZ_Brittney @RexChapman ...,@FoxNews But if the child accessed her gun and...,2955828382,RT @ISurveyMyDomain: @AZ_Brittney @RexChapman ...,False,NaN
3,Corona fake god told the Hobby Lobby guy's wif...,NaN,2020-03-22T01:41:31.000Z,@kgthatboy,KG,corona AND hobby lobby,kgthatboy,856715778099302401,KG,I'm worst at what I do best,False,Thu Apr 07 23:24:53 +0000 2022,@Onomatopathetic Damn might be a good idea I'l...,"RT @lolennui: 2022: this is a turning point, s...",856715778099302401,@Onomatopathetic Damn might be a good idea I'l...,False,I'm worst at what I do best
4,I wouldn't shop at Hobby Lobby if they were th...,NaN,2020-03-22T20:29:35.000Z,@jaguar_mick,Mick Jaguar,corona AND hobby lobby,jaguar_mick,970013930318802944,Mick Jaguar,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,False,Thu Apr 07 04:43:49 +0000 2022,"The birds and the ladybugs, right @LindseyGrah...",RT @carolecadwalla: We failed to acknowledge R...,970013930318802944,"The birds and the ladybugs, right @LindseyGrah...",False,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...


In [25]:
# counting unique values
n = len(pd.unique(joined['id_x']))
print(n)

6461


In [26]:
joined['verified'].value_counts()

False    6482
True      718
Name: verified, dtype: int64

In [27]:
joined = joined [["file", "user_name", "id_x","user_screen_name", "Embedded_text","description_x","verified","user_history" ]]

In [28]:
only_real_users = joined[joined['verified']==False]
only_real_users.shape

(6482, 8)

In [29]:
# Filter out user descriptions that dont exist and join the history + description option
df_desc = only_real_users[~only_real_users['description_x'].isnull()]
df_desc.shape

(5974, 8)

In [30]:
df_desc.head()

,file,user_name,id_x,user_screen_name,Embedded_text,description_x,verified,user_history
0,corona AND hobby lobby,rach,2639724316,rachelspires_,has hobby lobby shut down for this corona stuf...,girl mom ✨,False,RT @Closj_: bad bitches eat pickles|RT @mayowa...
1,corona AND hobby lobby,Gabriella Diamante de la Rosa,44444748,gabby_diamante,Companies need to do things correctly during t...,my personality shines bright and upfront on my...,False,When someone calls you a hater for stating fac...
3,corona AND hobby lobby,KG,856715778099302401,kgthatboy,Corona fake god told the Hobby Lobby guy's wif...,I'm worst at what I do best,False,"RT @lolennui: 2022: this is a turning point, s..."
4,corona AND hobby lobby,Mick Jaguar,970013930318802944,jaguar_mick,I wouldn't shop at Hobby Lobby if they were th...,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,False,RT @carolecadwalla: We failed to acknowledge R...
5,corona AND hobby lobby,Fatal Penguin,2869753558,yishai4121,"Hobby Lobby will still stay open, threatening ...","Carefully written, fact-checked essay in the s...",False,@nicolergalloway Worst leadership I've ever se...


In [31]:
df_desc.to_csv("CSA_2024/hao_input_data.csv", index=False, header=True)

In [32]:
# concatenate user history and description columns together
df_desc['user_context'] = df_desc['description_x']+df_desc['user_history']
df_desc.head()

<ipython-input-32-b3636400b8a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desc['user_context'] = df_desc['description_x']+df_desc['user_history']


,file,user_name,id_x,user_screen_name,Embedded_text,description_x,verified,user_history,user_context
0,corona AND hobby lobby,rach,2639724316,rachelspires_,has hobby lobby shut down for this corona stuf...,girl mom ✨,False,RT @Closj_: bad bitches eat pickles|RT @mayowa...,girl mom ✨RT @Closj_: bad bitches eat pickles|...
1,corona AND hobby lobby,Gabriella Diamante de la Rosa,44444748,gabby_diamante,Companies need to do things correctly during t...,my personality shines bright and upfront on my...,False,When someone calls you a hater for stating fac...,my personality shines bright and upfront on my...
3,corona AND hobby lobby,KG,856715778099302401,kgthatboy,Corona fake god told the Hobby Lobby guy's wif...,I'm worst at what I do best,False,"RT @lolennui: 2022: this is a turning point, s...",I'm worst at what I do bestRT @lolennui: 2022:...
4,corona AND hobby lobby,Mick Jaguar,970013930318802944,jaguar_mick,I wouldn't shop at Hobby Lobby if they were th...,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,False,RT @carolecadwalla: We failed to acknowledge R...,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...
5,corona AND hobby lobby,Fatal Penguin,2869753558,yishai4121,"Hobby Lobby will still stay open, threatening ...","Carefully written, fact-checked essay in the s...",False,@nicolergalloway Worst leadership I've ever se...,"Carefully written, fact-checked essay in the s..."


In [33]:
X_pred = df_desc['user_context']

In [34]:
y_predicted = load_model.predict(X_pred)
y_predicted = y_predicted.flatten()

187/187 [==============================] - 69s 341ms/step


In [35]:
df_desc['predicted_ideology_prob'] = y_predicted

<ipython-input-35-3eeb04e4eade>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desc['predicted_ideology_prob'] = y_predicted


In [36]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)

In [37]:
df_desc['predicted_ideology'] = y_predicted

<ipython-input-37-8c2741580c68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desc['predicted_ideology'] = y_predicted


In [38]:
df_desc.head()

,file,user_name,id_x,user_screen_name,Embedded_text,description_x,verified,user_history,user_context,predicted_ideology_prob,predicted_ideology
0,corona AND hobby lobby,rach,2639724316,rachelspires_,has hobby lobby shut down for this corona stuf...,girl mom ✨,False,RT @Closj_: bad bitches eat pickles|RT @mayowa...,girl mom ✨RT @Closj_: bad bitches eat pickles|...,0.588966,1
1,corona AND hobby lobby,Gabriella Diamante de la Rosa,44444748,gabby_diamante,Companies need to do things correctly during t...,my personality shines bright and upfront on my...,False,When someone calls you a hater for stating fac...,my personality shines bright and upfront on my...,0.599858,1
3,corona AND hobby lobby,KG,856715778099302401,kgthatboy,Corona fake god told the Hobby Lobby guy's wif...,I'm worst at what I do best,False,"RT @lolennui: 2022: this is a turning point, s...",I'm worst at what I do bestRT @lolennui: 2022:...,0.633113,1
4,corona AND hobby lobby,Mick Jaguar,970013930318802944,jaguar_mick,I wouldn't shop at Hobby Lobby if they were th...,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,False,RT @carolecadwalla: We failed to acknowledge R...,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,0.598017,1
5,corona AND hobby lobby,Fatal Penguin,2869753558,yishai4121,"Hobby Lobby will still stay open, threatening ...","Carefully written, fact-checked essay in the s...",False,@nicolergalloway Worst leadership I've ever se...,"Carefully written, fact-checked essay in the s...",0.778930,1


In [39]:
df_desc['predicted_ideology'].value_counts()

1    4458
0    1516
Name: predicted_ideology, dtype: int64

In [40]:
df_desc.shape

(5974, 11)

### Add sentiment analysis results as well ###
https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment
https://arxiv.org/abs/2104.12250

In [41]:
!pip install transformers

In [42]:
!pip install sentencepiece

In [43]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [45]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [46]:
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Pray from the safety of your home. That’s all"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
print(scores)

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


[0.09958787 0.59926677 0.3011454 ]


In [47]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
  #0th element is the most likely
assigned_label = config.id2label[ranking[0]]
print(assigned_label)

neutral


In [48]:
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.5993
2) positive 0.3011
3) negative 0.0996


### Running the Sentiment Analysis Model ###


In [49]:
res = df_desc

In [ ]:
# Say runtime expired, run this
#import pandas as pd
#res_path = "round3_modelrun_hao.csv"
#res = pd.read_csv(res_path, header=0)
#res.head()

In [50]:
def calculate_sentiment(row):
  text = row
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  a_list = list(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  #0th element is the most likely
  assigned_label = config.id2label[ranking[0]]
  a_list.append(assigned_label)
  return pd.Series(a_list)



In [51]:
res[['neg_sent','neutral_sent','pos_sent','assigned_sentiment_label']]=res['Embedded_text'].apply(calculate_sentiment)

<ipython-input-51-f5fddd66b1c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res[['neg_sent','neutral_sent','pos_sent','assigned_sentiment_label']]=res['Embedded_text'].apply(calculate_sentiment)
<ipython-input-51-f5fddd66b1c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res[['neg_sent','neutral_sent','pos_sent','assigned_sentiment_label']]=res['Embedded_text'].apply(calculate_sentiment)
<ipython-input-51-f5fddd66b1c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [52]:
res.head()

,file,user_name,id_x,user_screen_name,Embedded_text,description_x,verified,user_history,user_context,predicted_ideology_prob,predicted_ideology,neg_sent,neutral_sent,pos_sent,assigned_sentiment_label
0,corona AND hobby lobby,rach,2639724316,rachelspires_,has hobby lobby shut down for this corona stuf...,girl mom ✨,False,RT @Closj_: bad bitches eat pickles|RT @mayowa...,girl mom ✨RT @Closj_: bad bitches eat pickles|...,0.588966,1,0.685763,0.293394,0.020843,negative
1,corona AND hobby lobby,Gabriella Diamante de la Rosa,44444748,gabby_diamante,Companies need to do things correctly during t...,my personality shines bright and upfront on my...,False,When someone calls you a hater for stating fac...,my personality shines bright and upfront on my...,0.599858,1,0.903451,0.071776,0.024773,negative
3,corona AND hobby lobby,KG,856715778099302401,kgthatboy,Corona fake god told the Hobby Lobby guy's wif...,I'm worst at what I do best,False,"RT @lolennui: 2022: this is a turning point, s...",I'm worst at what I do bestRT @lolennui: 2022:...,0.633113,1,0.692709,0.227094,0.080197,negative
4,corona AND hobby lobby,Mick Jaguar,970013930318802944,jaguar_mick,I wouldn't shop at Hobby Lobby if they were th...,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,False,RT @carolecadwalla: We failed to acknowledge R...,Half Asian Leopard Cat. 1/4 Def Leppard cat. V...,0.598017,1,0.769016,0.182777,0.048207,negative
5,corona AND hobby lobby,Fatal Penguin,2869753558,yishai4121,"Hobby Lobby will still stay open, threatening ...","Carefully written, fact-checked essay in the s...",False,@nicolergalloway Worst leadership I've ever se...,"Carefully written, fact-checked essay in the s...",0.778930,1,0.923747,0.061222,0.015031,negative


In [53]:
res.shape

(5974, 15)

In [54]:
# counting unique values
n = len(pd.unique(res['id_x']))
print(n)

5426


In [55]:
res['assigned_sentiment_label'].value_counts()

negative    2861
neutral     2223
positive     890
Name: assigned_sentiment_label, dtype: int64

In [56]:
res['predicted_ideology'].value_counts()

1    4458
0    1516
Name: predicted_ideology, dtype: int64

### Get the human validation sample ###

In [59]:
# Filter the DataFrame for predicted_ideology == 1 and == 0
ideology_1 = res[res['predicted_ideology'] == 1].sample(n=150, random_state=42)
ideology_0 = res[res['predicted_ideology'] == 0].sample(n=150, random_state=42)

# Concatenate the two samples to get a balanced sample of 300
balanced_sample_ideology = pd.concat([ideology_1, ideology_0])

# Display or use the balanced sample
balanced_sample_ideology.to_csv("CSA_2024/ideology_300_sample.csv", header=True, index = False)

In [60]:
# Filter the DataFrame for each sentiment label and sample 100 from each
positive_sentiment = res[res['assigned_sentiment_label'] == 'positive'].sample(n=100, random_state=42)
negative_sentiment = res[res['assigned_sentiment_label'] == 'negative'].sample(n=100, random_state=42)
neutral_sentiment = res[res['assigned_sentiment_label'] == 'neutral'].sample(n=100, random_state=42)

# Concatenate the samples to get a balanced sample of 300
balanced_sample_sentiment = pd.concat([positive_sentiment, negative_sentiment, neutral_sentiment])

# Display or use the balanced sample
balanced_sample_sentiment.to_csv("CSA_2024/sentiment_300_sample.csv", header=True, index = False)

In [62]:
res.to_csv('CSA_2024/final_modelrun_hao_withsentimentandideology.csv',header=True, index = False)